In [62]:
import pandas as pd
import numpy as np
import spacy
from collections import Counter
import os
from IPython.display import display, HTML


## Compute a reference corpus

In [53]:
#comparison corpus german
from tei_reader import TeiReader
reader = TeiReader()
import os

dir_de_ref = 'swiss_history_books/reference_corpus_german/dta_kernkorpus_1700-1799_2020-07-20/'
dir_de_lemmatized = 'swiss_history_books/reference_corpus_german/dta_kernkorpus_1700-1799_2020-07-20_lemmatized/'

text_corpus_de = []
for files in os.listdir(dir_de_ref)[:100]:

    corpora = reader.read_file(dir_de_ref+files)
    
    ref_text = corpora.text.replace('ſ', 's')#replace ſ with s
    ref_text_cleaned = ' '.join([el for el in ref_text.split() if len(el) > 1])
    
    ref_text_lematized = lematization_pos_tagging(nlp_de, ref_text_cleaned[:1000000])#rate limit at 1 million limit texts
    text_corpus_de.append(ref_text_lematized)
    
    with open(dir_de_lemmatized+files[:-4]+".txt", "w") as text_file:
        text_file.write(' '.join(ref_text_lematized))
    


In [54]:
#comparison corpus french
from tei_reader import TeiReader
reader = TeiReader()


dir_fr_ref = 'swiss_history_books/reference_corpus_french/17_xml/'
dir_fr_lemmatized = 'swiss_history_books/reference_corpus_french/17_galica_lemmatized/'


text_corpus_de = []
for files in os.listdir(dir_fr_ref)[:100]:
    
    corpora = reader.read_file(dir_fr_ref+files)
    ref_text = corpora.text

    ref_text_cleaned = ' '.join([el for el in ref_text.split() if len(el) > 1])
    
    ref_text_lematized = lematization_pos_tagging(nlp_fr, ref_text_cleaned[:1000000])#rate limit at 1 million limit texts
    text_corpus_de.append(ref_text_lematized)
    
    with open(dir_fr_lemmatized+files[:-4]+".txt", "w") as text_file:
        text_file.write(' '.join(ref_text_lematized))
        
        

## Read in swiss history books 

In [26]:
#read in
path= '/home/nulpe/Desktop/H-D_Project/swiss_history_books/german_swiss_nation_building_texts/'
ch_dir = [path+ 'Der_Geschichten_schweizerischer_Eidgenos.txt', path+'Die Schweiz im neunzehnten Jahrhundert.txt']# ,[path+'Aegidii_Tschudii_gewesenen_Land_Ammanns.txt'] 
fr_dir = [path+'Histoire_de_la_Confédération_Suisse-1.txt', path+'la_suisse_19_siecle.txt']

lst_schweizer_geschichte = []
for el in ch_dir:
    with open(el) as f:
        schweizer_geschichte = f.read()
    lst_schweizer_geschichte.append(schweizer_geschichte)


lst_confederation_suisse = []
for el in fr_dir:
    with open(el) as f:
        confederation_suisse = f.read()
    lst_confederation_suisse.append(confederation_suisse)



In [27]:
# preprocessing
pos_to_keep = ['ADJ', 'ADV', 'NOUN', 'PROPN', 'VERB']

def lematization_pos_tagging(tagging_language_funtion, text):
    doc = tagging_language_funtion(text)
    lemmatized = [x.lemma_ for x in doc if x.pos_ in pos_to_keep and x.is_stop != True and len(x.lemma_)>1]
    print()
    return lemmatized

#load spacy corpuses
nlp_de = spacy.load('de_core_news_md')
nlp_fr = spacy.load('fr_core_news_sm')

In [28]:
#lemmatize german and french version

for idx, schweizer_geschichte in enumerate(lst_schweizer_geschichte):
    schweizer_geschichte = schweizer_geschichte.replace('ſ', 's')#replace ſ with s
    schweizer_geschichte_lematized = lematization_pos_tagging(nlp_de, schweizer_geschichte[:1000000])
    #schweizer_tf = Counter(schweizer_geschichte_lematized)
    lst_schweizer_geschichte[idx]=schweizer_geschichte_lematized
    
for idx, confederation_suisse in enumerate(lst_confederation_suisse):    
    confederation_suisse_lemmatized = lematization_pos_tagging(nlp_fr, confederation_suisse[:1000000])
    #confederation_tf = Counter(confederation_suisse_lemmatized)
    lst_confederation_suisse[idx]=confederation_suisse_lemmatized



In [29]:
#pre processing with lematization

#print(schweizer_tf)
#print((confederation_tf))
lst_schweizer_geschichte_final = ''
for el in lst_schweizer_geschichte: lst_schweizer_geschichte_final+=' '.join(el) 
#lst_schweizer_geschichte_final = [' '.join(el) for el in lst_schweizer_geschichte_final]
schweizer_tf = Counter(lst_schweizer_geschichte_final)
lst_confederation_suisse_final = [' '.join(el) for el in lst_confederation_suisse]
confederation_tf = Counter(lst_confederation_suisse_final)

schweizer_tf_top_100 = [(el, schweizer_tf[el])   for  el in schweizer_tf]
schweizer_tf_top_100 = sorted(schweizer_tf_top_100, key=lambda tup: tup[1], reverse=True)
confederation_tf_top_100 = [(el, confederation_tf[el])  for el in confederation_tf]
confederation_tf_top_100 = sorted(confederation_tf_top_100, key=lambda tup: tup[1], reverse=True)


#print(schweizer_tf_top_100[:100])
#print(confederation_tf_top_100[:100])

print(len(lst_schweizer_geschichte_final))


# --> join again the list of strings 

1053968


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
dir_de_lemmatized = 'swiss_history_books/reference_corpus_german/dta_kernkorpus_1700-1799_2020-07-20_lemmatized/'
dir_fr_lemmatized = 'swiss_history_books/reference_corpus_french/17_galica_lemmatized/'

def ref_corp(dir_lemmatized):
    text_corpus = []
    for path in os.listdir(dir_lemmatized):
        with open(dir_lemmatized+path) as f:
             text_corpus.append(f.read())
    return text_corpus

def k_gram_corpus(text_corpus, k_gram):
    bigram_corpus = []
    for doc in text_corpus:
        gram_doc = ' '.join([el[0]+'_'+el[1]  for el in nltk.ngrams(doc.split(), 2) if len(el[0]) > 3 and len(el[1]) > 3 ])
        if k_gram == 3: gram_doc+= ' '.join([el[0]+'_'+el[1]+'_'+el[2]   for el in nltk.ngrams(doc.split(), k_gram) if len(el[0]) > 3 and len(el[1]) > 3 ])
        bigram_corpus.append(gram_doc)
    return bigram_corpus


#corpus in fr and de
text_corpus_de = ref_corp(dir_de_lemmatized)
text_corpus_fr = ref_corp(dir_fr_lemmatized)
lst_schweizer_geschichte_final = [' '.join(el) for el in lst_schweizer_geschichte]

lst_schweizer_geschichte_final = ''
for el in lst_schweizer_geschichte: lst_schweizer_geschichte_final+=' '.join(el) 

lst_confederation_suisse_final = [' '.join(el) for el in lst_confederation_suisse]
text_corpus_de.append(lst_schweizer_geschichte_final)
text_corpus_fr.append(' '.join(lst_confederation_suisse_final))



#Unigrams 
vectorizer_de = TfidfVectorizer()
vectorizer_fr = TfidfVectorizer()
de = vectorizer_de.fit_transform(text_corpus_de)
fr = vectorizer_fr.fit_transform(text_corpus_fr)


#Bigrams
text_corpus_de_bigram = k_gram_corpus(text_corpus_de, 3) 
text_corpus_fr_bigram = k_gram_corpus(text_corpus_fr, 3)



vectorizer_de_bi = TfidfVectorizer(token_pattern=r'\S+')
vectorizer_fr_bi = TfidfVectorizer(token_pattern=r'\S+')
de_bi = vectorizer_de_bi.fit_transform(text_corpus_de_bigram)
fr_bi = vectorizer_fr_bi.fit_transform(text_corpus_fr_bigram)


#ToDo:
#fix bigrams....
#add german french history texts 

In [31]:

for el in text_corpus_fr_bigram[50].split():
    if '_'  not in el:
        print(el)
        
print('\n')

#bigram_corpus.append(' '.join([doc.split()[idx]+'_'+doc.split()[idx+1]  for idx in range(len(doc.split())-1)]))

print(len(text_corpus_de), len(text_corpus_de_bigram))
print(len(text_corpus_fr_bigram), len(text_corpus_fr_bigram))



101 101
101 101


In [38]:
def make_tfidf_table(matrix, vectorizer_lang):
    tfidf = pd.DataFrame(matrix[-1].toarray().T)
    tfidf.index = vectorizer_lang.get_feature_names()
    return tfidf.sort_values(by = 0, ascending=False).reset_index()



schweizer_geschichte_tfidf = make_tfidf_table(de, vectorizer_de)
confederation_suisse_tfidf = make_tfidf_table(fr, vectorizer_fr)

schweizer_geschichte_tfidf_bi = make_tfidf_table(de_bi, vectorizer_de_bi)
confederation_suisse_tfidf_bi = make_tfidf_table(fr_bi, vectorizer_fr_bi)

print(pd.concat([confederation_suisse_tfidf[:40],  confederation_suisse_tfidf_bi[:40]], axis=1)) 
print(pd.concat([schweizer_geschichte_tfidf[:40],schweizer_geschichte_tfidf_bi[:40]], axis=1))


          index         0                   index         0
0      rodolphe  0.332392            comte_pierre  0.094148
1         comte  0.245470          comte_rodolphe  0.081072
2       tschudi  0.221161            comte_savoie  0.071237
3         berne  0.198825             ville_berne  0.054140
4         ville  0.158460         abbé_saint-gall  0.054140
5        zurich  0.147188        maison_habsbourg  0.051291
6        albert  0.119029             marc_argent  0.048722
7            ch  0.117110      rodolphe_habsbourg  0.048441
8        suisse  0.116733           comte_palatin  0.047074
9     bourgeois  0.115486          comte_eberhard  0.042742
10     seigneur  0.114379         bourgeois_berne  0.042742
11          roi  0.107952          rodolphe_broun  0.039893
12    habsbourg  0.107745          avoyer_conseil  0.039893
13         jean  0.107196          jean_bubenberg  0.039893
14         pays  0.106166          empereur_henri  0.039228
15      château  0.105915          emper

## N-Grams Impresson & Latex conform print out

In [66]:
print(' '.join(schweizer_geschichte_tfidf['index'][:40].tolist()))

kanton schweiz regierung truppe französisch zürich tagsatzung partei frankreich eidgenössisch helvetischen behörde luzern droz eidgenossenschaft verfassung bern waadt helvetische bürger schweizerisch öffentlich verninac erklären französische consul wallis juli märz schwyz republik bundesrat freiburg helvetien volk genf neutralität mehrheit abgeordnete basel


In [67]:
print(' '.join(confederation_suisse_tfidf['index'][:40].tolist()))

rodolphe comte tschudi berne ville zurich albert ch suisse bourgeois seigneur roi habsbourg jean pays château charte évêque waldstetten kibourg guerre hartmann bernois maison schwyz abbé empereur saint savoie henri chevalier grand neuchâtel duc fribourg gall eberhard bâle chron pierre


In [51]:
lst_bigrams = []
for el in schweizer_geschichte_tfidf_bi['index'][:25].tolist(): lst_bigrams+= el.split('_') 
print(' '.join(lst_bigrams))

numa droz französisch truppe helvetische regierung helvetischen republik droz wiedergeburt öffentlich meinung droz protektorat bull lois helvetische republik helvetischen regierung französisch regierung ordnung ding regie rung kanton bern französische regierung vollz nathes kanton waadt prot vollz zürich bern demokratisch kanton protof vollz numa droz wiedergeburt verfassung geben lluma droz numa droz protektorat


In [52]:
lst_bigrams = []
for el in confederation_suisse_tfidf_bi['index'][:25].tolist(): lst_bigrams+= el.split('_') 
print(' '.join(lst_bigrams))

comte pierre comte rodolphe comte savoie marc argent abbé saint-gall ville berne maison habsbourg comte palatin rodolphe habsbourg empereur henri comte eberhard bourgeois berne jean bubenberg rodolphe broun avoyer conseil empereur louis comte habsbourg comte amédée bourgeois ville comte kibourg comte hartmann pierre gruyère suisse allemand cent marc empereur frédéric


In [64]:
print(pd.concat([pd.Series(['&']*40), confederation_suisse_tfidf['index'][:40],pd.Series(['&']*40), confederation_suisse_tfidf[0][:40],pd.Series(['&']*40), 
                 confederation_suisse_tfidf_bi[:40]['index'].apply(lambda x: x.replace('_', ' ')),pd.Series(['&']*40), confederation_suisse_tfidf_bi[0][:40], pd.Series(['\\\\']*40)], axis=1))




    0        index  1         0  2                   index  3         0   4
0   &     rodolphe  &  0.332392  &            comte pierre  &  0.094148  \\
1   &        comte  &  0.245470  &          comte rodolphe  &  0.081072  \\
2   &      tschudi  &  0.221161  &            comte savoie  &  0.071237  \\
3   &        berne  &  0.198825  &             ville berne  &  0.054140  \\
4   &        ville  &  0.158460  &         abbé saint-gall  &  0.054140  \\
5   &       zurich  &  0.147188  &        maison habsbourg  &  0.051291  \\
6   &       albert  &  0.119029  &             marc argent  &  0.048722  \\
7   &           ch  &  0.117110  &      rodolphe habsbourg  &  0.048441  \\
8   &       suisse  &  0.116733  &           comte palatin  &  0.047074  \\
9   &    bourgeois  &  0.115486  &          comte eberhard  &  0.042742  \\
10  &     seigneur  &  0.114379  &         bourgeois berne  &  0.042742  \\
11  &          roi  &  0.107952  &          rodolphe broun  &  0.039893  \\
12  &    hab

In [63]:

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n","<br>")))

pretty_print(pd.concat([pd.Series(['&']*40), schweizer_geschichte_tfidf['index'][:40],pd.Series(['&']*40), schweizer_geschichte_tfidf[0][:40],pd.Series(['&']*40), 
                 schweizer_geschichte_tfidf_bi[:40]['index'].apply(lambda x: x.replace('_', ' ')),pd.Series(['&']*40), schweizer_geschichte_tfidf_bi[0][:40], pd.Series(['\\\\']*40)], axis=1))



,0,index,1,0,2,index,3,0,4
0,&,kanton,&,0.517982,&,numa droz,&,0.113806,\\
1,&,schweiz,&,0.314341,&,französisch truppe,&,0.096147,\\
2,&,regierung,&,0.165484,&,helvetische regierung,&,0.090260,\\
3,&,truppe,&,0.153276,&,helvetischen republik,&,0.074563,\\
4,&,französisch,&,0.146008,&,droz wiedergeburt,&,0.068676,\\
5,&,zürich,&,0.143091,&,öffentlich meinung,&,0.066714,\\
6,&,tagsatzung,&,0.127124,&,droz protektorat,&,0.064752,\\
7,&,partei,&,0.126443,&,bull lois,&,0.062790,\\
8,&,frankreich,&,0.107725,&,helvetische republik,&,0.056903,\\
9,&,eidgenössisch,&,0.106244,&,helvetischen regierung,&,0.056903,\\


## Time trends Impresso

In [ ]:
gdl_trend = 'ngrams_trends/unigrams_confederation.json'
nzz_trend = 'ngrams_trends/unigrams_eidgenossenschaft.json'